Coder: Gabriel Levin

Date: 1/30/2022

This code extracts health, flood, and drought data from the follow data sources, runs zonal statistics on the Payam Lvl, and exports the data into an excel file:
- Livelihood 
- Stunting
- Underweight
- Wasting
- Severe Wasting
- SPEI
- SPI

In [1]:
import sys, os
import geopandas as gpd
import rasterio as rio
import numpy as np
from rasterstats import zonal_stats
from rasterio.plot import show
import pandas as pd
from glob import glob
import statistics
from tqdm import tqdm

In [2]:
data_dir = r"D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\RawData\\"

In [3]:
zones = gpd.read_file(data_dir+ "\AdminBoundaries\SS_ADM3_clipped.shp")

# 1. Livelihood Zones

In [4]:
live_zones = gpd.read_file(data_dir+ "\LivelihoodZones\SS_LHZ_2018.shp")

In [5]:
live_zones

,FNID,EFF_YEAR,COUNTRY,LZNUM,LZCODE,LZNAMEEN,CLASS,geometry
0,SS2018L101,2018,SS,1,SS01,Equatorial maize and cassava,AN02,"MULTIPOLYGON (((28.70507 5.53918, 28.73123 5.5..."
1,SS2018L102,2018,SS,2,SS02,Ironstone Plateau agro-pastoral,AN07,"POLYGON ((30.81773 6.79966, 30.83221 6.78663, ..."
2,SS2018L104,2018,SS,4,SS04,Western plains groundnuts sesame and sorghum,AG08,"POLYGON ((26.21867 9.89578, 26.21867 9.89578, ..."
3,SS2018L105,2018,SS,5,SS05,South-Eastern semi-arid pastoral,SH02,"POLYGON ((33.22558 7.05603, 33.22672 7.05318, ..."
4,SS2018L106,2018,SS,6,SS06,Eastern plains sorghum and cattle,AS09,"POLYGON ((31.79547 9.22550, 31.79989 9.20783, ..."
5,SS2018L107,2018,SS,7,SS07,Greater Bahr el Ghazal sorghum and cattle,AS05,"POLYGON ((29.00037 9.67023, 29.00037 9.67023, ..."
6,SS2018L108,2018,SS,8,SS08,Nile basin fishing and agro-pastoral,FS02,"POLYGON ((30.75601 9.49297, 30.75581 9.49248, ..."
7,SS2018L109,2018,SS,9,SS09,North-Western Nile basin cattle and maize,CC02,"POLYGON ((30.18830 9.46424, 30.18802 9.43906, ..."
8,SS2018L110,2018,SS,10,SS10,North-Eastern maize cattle and fishing,PS02,"POLYGON ((33.89957 9.49864, 33.90179 9.49869, ..."
9,SS2018L111,2018,SS,11,SS11,Northern sorhgum and livestock,PS04,"POLYGON ((33.89957 9.49864, 33.89957 9.49864, ..."


In [6]:
live_zones = live_zones.drop(columns=['EFF_YEAR', 'COUNTRY', 'LZNUM', 'LZCODE', 'CLASS'])

In [8]:
zones.to_file(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme4\temp\zones1.shp')

In [9]:
zones.columns

Index(['Shape_Leng', 'Shape_Area', 'ADM3_EN', 'ADM3_PCODE', 'ADM3_REF',
       'ADM3ALT1EN', 'ADM3ALT2EN', 'ADM2_EN', 'ADM2_PCODE', 'ADM1_EN',
       'ADM1_PCODE', 'ADM0_EN', 'ADM0_PCODE', 'date', 'validOn', 'validTo',
       'geometry'],
      dtype='object')

In [10]:
ol_zones = gpd.overlay(zones, live_zones, how='intersection')

In [11]:
ol_zones.to_file(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme4\temp\zones2.shp')

In [12]:
ol_zones.sort_values(by='geometry', inplace=True, key =lambda col: np.array([x.area for x in col]))

In [13]:
ol_zones.drop_duplicates(subset='ADM3_EN', keep='last', inplace=True)

In [14]:
ol_zones= ol_zones.rename({'FNID':'LZFNID'}, axis = 1)

In [16]:
zones = zones.merge(ol_zones[['ADM3_PCODE','LZFNID','LZNAMEEN']], on= 'ADM3_PCODE' )

In [18]:
zones.to_file(r'D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme4\temp\zones3.shp')

# 2. Health Data Aggregations

In [19]:
stunting_dir = data_dir + r"\Stunting\1 - Under 5"
underweight_dir = data_dir + r"\Underweight\1 - Under 5"
wasting_dir = data_dir + r"\Wasting\1 - Under 5"
severewasting_dir = data_dir + r"\SevereWasting\1 - Under 5"

healthdata_dir = [stunting_dir,underweight_dir,wasting_dir,severewasting_dir]
healthdata_name = ['STUNTING','UNDERWEIGHT','WASTING','SEVERE_WASTING']
healthdata_lable = ['ST','UW','Wa','SW']

In [20]:
healthdata_S3_mean = glob(healthdata_dir[0] + '/IHME_GLOBAL_CGF_2000_2019_' + healthdata_name[0] + '_PREV_PERCENT_A1_S3_MEAN*')

In [21]:
for d in range(len(healthdata_dir)):
    healthdata_S3_mean = glob(healthdata_dir[d] + '/IHME_GLOBAL_CGF_2000_2019_' + healthdata_name[d] + '_PREV_PERCENT_A1_S3_MEAN*')
    for i in healthdata_S3_mean:
        test_rio = rio.open(i)
        data = test_rio.read(1)
        data[np.where(data == -999999.0)] = 0
        zs = zonal_stats(zones, data, affine=test_rio.transform, stats='mean', nodata=0) #Zonal Statistics

        meanONLY = []
        for num in zs:
            meanONLY.append(num['mean']) #Extract list of numbers only

        col_name_lst = (i.split("A1_",1)[1].split("_",3))
        col_name = healthdata_lable[d] + '_' + col_name_lst[0] + '_' + col_name_lst[1] + '_' + col_name_lst[2]
        print (col_name)
        zones[col_name] = meanONLY

ST_S3_MEAN_2000
ST_S3_MEAN_2001
ST_S3_MEAN_2002
ST_S3_MEAN_2003
ST_S3_MEAN_2004
ST_S3_MEAN_2005
ST_S3_MEAN_2006
ST_S3_MEAN_2007
ST_S3_MEAN_2008
ST_S3_MEAN_2009
ST_S3_MEAN_2010
ST_S3_MEAN_2011
ST_S3_MEAN_2012
ST_S3_MEAN_2013
ST_S3_MEAN_2014
ST_S3_MEAN_2015
ST_S3_MEAN_2016
ST_S3_MEAN_2017
ST_S3_MEAN_2018
ST_S3_MEAN_2019
UW_S3_MEAN_2000
UW_S3_MEAN_2001
UW_S3_MEAN_2002
UW_S3_MEAN_2003
UW_S3_MEAN_2004
UW_S3_MEAN_2005
UW_S3_MEAN_2006
UW_S3_MEAN_2007
UW_S3_MEAN_2008
UW_S3_MEAN_2009
UW_S3_MEAN_2010
UW_S3_MEAN_2011
UW_S3_MEAN_2012
UW_S3_MEAN_2013
UW_S3_MEAN_2014
UW_S3_MEAN_2015
UW_S3_MEAN_2016
UW_S3_MEAN_2017
UW_S3_MEAN_2018
UW_S3_MEAN_2019
Wa_S3_MEAN_2000
Wa_S3_MEAN_2001
Wa_S3_MEAN_2002
Wa_S3_MEAN_2003
Wa_S3_MEAN_2004
Wa_S3_MEAN_2005
Wa_S3_MEAN_2006
Wa_S3_MEAN_2007
Wa_S3_MEAN_2008
Wa_S3_MEAN_2009
Wa_S3_MEAN_2010
Wa_S3_MEAN_2011
Wa_S3_MEAN_2012
Wa_S3_MEAN_2013
Wa_S3_MEAN_2014
Wa_S3_MEAN_2015
Wa_S3_MEAN_2016
Wa_S3_MEAN_2017
Wa_S3_MEAN_2018
Wa_S3_MEAN_2019
SW_S3_MEAN_2000
SW_S3_MEAN_2001
SW_S3_ME

# 3. Drought

### 12 Month Mean of Means

In [22]:
drought_dir = r"E:\TempWorking\SSD_Water\Drought\SPEI"

In [23]:
yr_list = list(range(0, 21))
lstoflst = []
lst_means = []
for yr in yr_list:
    if yr <10:
        yr_str = '0' + str(yr)
    elif yr >=10:
        yr_str = str(yr)
        
    drought_files_yr = glob(drought_dir + '/wld_cli_spei_gamma_12_terraclimate_20'+ yr_str +'*'+'.tif')
    yr += 1
    print(yr)
    
    lstoflst_yr = []
    for data in drought_files_yr:
        lyr_rio = rio.open(data)
        data = lyr_rio.read(1)
        data[np.where(data == -999999.0)] = 0
        zs = zonal_stats(zones, data, affine=lyr_rio.transform, stats='mean', nodata=0)
        
        meanONLY = []
        for num in zs:
            meanONLY.append(num['mean'])
        
        lstoflst_yr.append(meanONLY)
        meanONLY_arr=np.nanmean(np.array(lstoflst_yr,dtype=float), axis=0) #convert to np array to use np.average across rows(axis =1)
    
    lstoflst.append(list(meanONLY_arr))

df = pd.DataFrame(lstoflst).T
df_final = df.rename(columns=lambda s: "DR_MEAN_" + str(s+2000))
zones = pd.concat([zones, df_final], axis=1, join="inner")

1


C:\Users\glev2\AppData\Local\Temp/ipykernel_36848/2919178626.py:26: RuntimeWarning: Mean of empty slice
  meanONLY_arr=np.nanmean(np.array(lstoflst_yr,dtype=float), axis=0) #convert to np array to use np.average across rows(axis =1)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [24]:
type(zones)

geopandas.geodataframe.GeoDataFrame

In [25]:
#outdir = r"D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme4\SS_ADM3_clipped_wHealthMetrics_wDroughtByYear.shp"
#zones.to_file(outdir)

In [26]:
#zones.to_excel(r"D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme4\SS_ADM3_clipped_wHealthMetrics_wDroughtByYear.xlsx")

# 4. Flood SPI

In [27]:
# outdir = r"D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme4\SS_ADM3_clipped_wHealthMetrics_wDroughtByYear.xlsx"
# zones_wData =gpd.read_excel(outdir)


In [28]:
zones

,Shape_Leng,Shape_Area,ADM3_EN,ADM3_PCODE,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,ADM2_EN,ADM2_PCODE,ADM1_EN,...,DR_MEAN_2011,DR_MEAN_2012,DR_MEAN_2013,DR_MEAN_2014,DR_MEAN_2015,DR_MEAN_2016,DR_MEAN_2017,DR_MEAN_2018,DR_MEAN_2019,DR_MEAN_2020
0,1.096062,0.054905,Bungu,SS010101,None,None,None,Juba,SS0101,Central Equatoria,...,0.055137,-1.204098,-1.703529,-1.689911,-2.399144,-1.993250,-1.515015,-1.626102,-1.717998,-1.368542
1,1.791721,0.149619,Dolo,SS010102,None,None,None,Juba,SS0101,Central Equatoria,...,0.013953,-1.282194,-1.951851,-1.809827,-2.194494,-2.047694,-2.038466,-2.013429,-1.964985,-1.445876
2,0.762490,0.029053,Ganji,SS010103,None,None,None,Juba,SS0101,Central Equatoria,...,0.054102,-1.198984,-1.680912,-1.717427,-2.455266,-1.961614,-1.427746,-1.549149,-1.681734,-1.179175
3,1.232994,0.051823,Gondokoro,SS010104,None,None,None,Juba,SS0101,Central Equatoria,...,0.072561,-1.048741,-1.394399,-1.383111,-2.078260,-1.641418,-1.352007,-1.495276,-1.844872,-1.650606
4,0.161833,0.001406,Juba Town,SS010105,None,None,None,Juba,SS0101,Central Equatoria,...,0.081442,-1.127010,-1.632322,-1.535256,-2.145673,-1.742766,-1.449767,-1.606156,-1.763258,-1.573203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,1.319038,0.049060,Ngamunde,SS100305,None,None,None,Maridi,SS1003,Western Equatoria,...,0.048833,-1.566867,-2.147868,-1.897899,-2.104019,-2.079141,-2.705305,-2.336024,-2.039960,-1.708137
507,1.033285,0.048474,Gangura,SS101002,None,None,None,Yambio,SS1010,Western Equatoria,...,0.086032,-1.692300,-2.219153,-2.029029,-2.216822,-2.082566,-2.473125,-2.153060,-2.188025,-1.818624
508,1.963917,0.228002,Mambe,SS100303,None,None,None,Maridi,SS1003,Western Equatoria,...,0.018193,-1.535563,-2.141030,-1.861302,-2.060256,-2.074923,-2.676703,-2.357133,-2.004591,-1.727854
509,1.163445,0.040036,Sakure,SS100804,None,None,None,Nzara,SS1008,Western Equatoria,...,0.091015,-1.738036,-2.229870,-2.036309,-2.224853,-2.101720,-2.435743,-2.117405,-2.168383,-1.767148


In [29]:
SPI_dir = r"E:\TempWorking\SSD_Water\SLI\geotiffs"

In [30]:
yr_list = list(range(2000, 2021))
lstoflst = []
lst_means = []
for yr in yr_list:
        
    drought_files_yr = glob(SPI_dir + '/ssd_cli_chirps_spi12_'+ str(yr) +'*'+'.tif')
    yr += 1
    print(yr)
    
    lstoflst_yr = []
    for data in drought_files_yr:
        lyr_rio = rio.open(data)
        data = lyr_rio.read(1)
        data[np.where(data == -999999.0)] = 0
        zs = zonal_stats(zones, data, affine=lyr_rio.transform, stats='mean', nodata=0)
        
        meanONLY = []
        for num in zs:
            meanONLY.append(num['mean'])
        
        lstoflst_yr.append(meanONLY)
        meanONLY_arr=np.nanmean(np.array(lstoflst_yr,dtype=float), axis=0) #convert to np array to use np.average across rows(axis =1)
    
    lstoflst.append(list(meanONLY_arr))

df = pd.DataFrame(lstoflst).T
df_final = df.rename(columns=lambda s: "SPI_MEAN_" + str(s+2000))
zones = pd.concat([zones, df_final], axis=1, join="inner")

2001


C:\Users\glev2\AppData\Local\Temp/ipykernel_36848/185543014.py:26: RuntimeWarning: Mean of empty slice
  meanONLY_arr=np.nanmean(np.array(lstoflst_yr,dtype=float), axis=0) #convert to np array to use np.average across rows(axis =1)


2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [31]:
zones

,Shape_Leng,Shape_Area,ADM3_EN,ADM3_PCODE,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,ADM2_EN,ADM2_PCODE,ADM1_EN,...,SPI_MEAN_2011,SPI_MEAN_2012,SPI_MEAN_2013,SPI_MEAN_2014,SPI_MEAN_2015,SPI_MEAN_2016,SPI_MEAN_2017,SPI_MEAN_2018,SPI_MEAN_2019,SPI_MEAN_2020
0,1.096062,0.054905,Bungu,SS010101,None,None,None,Juba,SS0101,Central Equatoria,...,0.028953,0.446211,0.907365,0.504841,0.203953,0.646027,-0.130390,0.423833,0.346399,0.650118
1,1.791721,0.149619,Dolo,SS010102,None,None,None,Juba,SS0101,Central Equatoria,...,0.141306,0.412851,0.794300,0.391628,0.078700,0.774975,-0.407142,0.138447,0.164517,0.386991
2,0.762490,0.029053,Ganji,SS010103,None,None,None,Juba,SS0101,Central Equatoria,...,0.170319,0.534561,0.750864,0.397243,0.021424,0.388093,-0.175939,0.394174,0.382143,0.785723
3,1.232994,0.051823,Gondokoro,SS010104,None,None,None,Juba,SS0101,Central Equatoria,...,0.359532,0.618421,0.859209,0.588625,0.335352,0.395179,-0.284357,0.597523,0.391932,0.751026
4,0.161833,0.001406,Juba Town,SS010105,None,None,None,Juba,SS0101,Central Equatoria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,1.319038,0.049060,Ngamunde,SS100305,None,None,None,Maridi,SS1003,Western Equatoria,...,0.428800,1.266780,1.235820,0.628549,0.115998,0.345425,-0.366993,0.304917,-0.075915,0.253437
507,1.033285,0.048474,Gangura,SS101002,None,None,None,Yambio,SS1010,Western Equatoria,...,-0.616015,0.371562,1.778239,0.889636,-0.046400,0.045349,-1.007459,-0.203570,0.198105,-0.168444
508,1.963917,0.228002,Mambe,SS100303,None,None,None,Maridi,SS1003,Western Equatoria,...,0.350387,1.173172,1.156632,0.438464,-0.127652,0.071905,-0.629678,0.261023,-0.065524,0.031894
509,1.163445,0.040036,Sakure,SS100804,None,None,None,Nzara,SS1008,Western Equatoria,...,-0.773783,0.159345,1.924238,1.172435,-0.093054,0.046390,-1.160073,-0.340570,0.245842,-0.194309


In [33]:
zones.to_excel(r"D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme4\Theme4_Health_Flood_Drought_byPayam.xlsx")

In [34]:
zones.to_file(r"D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme4\Theme4_Health_Flood_Drought_byPayam.shp")

C:\Users\glev2\AppData\Local\Temp/ipykernel_36848/2881304842.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  zones.to_file(r"D:\OneDrive\Work\WorldBank\Assignments\20211104_SSudanWaterSecurity\Data\Theme4\Theme4_Health_Flood_Drought_byPayam.shp")
